In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.io import loadmat
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import re


In [2]:
def plotData(X , y ,S):
    pos= (y==1).ravel()
    neg= (y==0).ravel()

    plt.scatter(X[pos,0], X[pos,1], s=S, c='b', marker='+', linewidths=1)
    plt.scatter(X[neg,0], X[neg,1], s=S, c='r', marker='o', linewidths=1)


def plot_svc(svc, X, y, h=0.02, pad=0.25):
    x_min, x_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    y_min, y_max = X[:, 1].min()-pad, X[:, 1].max()+pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = svc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)

    plotData(X, y, 6)
    # plt.scatter(X[:, 0], X[:, 1], s=70, c=y, cmap=plt.cm.Paired)
    # Support vectors indicated in plot by vertical lines
    sv = svc.support_vectors_
    plt.scatter(sv[:, 0], sv[:, 1], c='y', marker='|', s=100, linewidths='5')
    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.show()


In [3]:
split_df =  pd.read_csv('split_df.csv',encoding='utf-8-sig')

In [4]:
X_data = split_df.drop(['Diagnosis','Table 1','Gender','id','Name','Eye','Age','Spectacles'],axis=1)
y_data = split_df['Diagnosis']

In [5]:
X_data

,VA,CDR,IOP
0,0.6,0.5,14.0
1,0.7,0.9,23.0
2,NaN,NaN,25.0
3,9.0,0.8,40.0
4,1.0,0.5,15.0
...,...,...,...
85,NaN,NaN,NaN
86,0.3,0.5,15.0
87,0.6,0.7,16.0
88,0.5,0.3,15.0


In [6]:
# imputer = SimpleImputer(strategy='mean')
# X_data_imputed = imputer.fit_transform(X_data)

In [7]:
# plotData(X_data_imputed,y_data,5)   

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [9]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [10]:
clf = svm.SVC(C=0.9,kernel='linear')

clf.fit(X_train_imputed,y_train)

y_pred = clf.predict(X_test_imputed)

# حساب الدقة
accuracy = accuracy_score(y_test, y_pred)

# طباعة الدقة
print(f'accuracy: {accuracy}')

accuracy: 0.6666666666666666


In [11]:
# clf = svm.SVC(C=0.9,kernel='rbf',gamma=2)
clf.set_params(C=1,kernel='rbf',gamma=2)## Gamma مدى الليونة

clf.fit(X_train_imputed,y_train)

y_pred = clf.predict(X_test_imputed)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.4444444444444444


In [12]:
# clf = svm.SVC(C=1.0,kernel='poly',degree= 3,gamma=3)
clf.set_params(C=1.0,kernel='poly',degree= 3,gamma=2)

clf.fit(X_train_imputed,y_train)

y_pred = clf.predict(X_test_imputed)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.5555555555555556


In [13]:
clf2 = svm.SVC()

clf2.fit(X_train_imputed,y_train)

y_pred = clf2.predict(X_test_imputed)

accuracy = accuracy_score(y_test, y_pred)

print(f'accuracy: {accuracy}')

accuracy: 0.7777777777777778
